## Agent

Agent是智能体，也称为代理，它可以帮你做出决策

代理的核心思想是使用语言模型来选择要执行的操作序列。在链中，操作序列是硬编码的(在代码中)。在代理中，语言模型被用作一个推理引擎来决定采取哪些行动以及按照哪个顺序。

In [1]:
# 环境变量设置
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"

### 基础示例

下面的示例会构建一个具有两个工具的代理: 

- 一个用于在线查找

- 另一个用于查找加载到索引中的特定数据

#### 定义工具tools

##### Tavily

Tavily是在线搜索的api，设置对应的api_key（需要先去官网注册获取api_key）

**LangChain 有一个内置的工具，可以轻松地使用 Tavily 搜索引擎作为工具**，下面是在langchain中的简单使用

In [2]:
os.environ["TAVILY_API_KEY"] = "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"

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
search = TavilySearchResults()

In [ ]:
search.invoke("今天北京的天气怎么样？")

##### Retriever

处理Tavily在线搜索的工具之外，再创建一个检索特定内容的工具

下面通过加载es6 promise的一篇文档作为检索器的搜索内容

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://es6.ruanyifeng.com/#docs/promise")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [ ]:
retriever.invoke("Es6 Promise怎么捕获错误")

将检索器转换为工具

In [6]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "es6_promise_search",
    "This is an introduction to the use of es6 promise. For any questions about ES6 Promise, you must use this tool!",
)

##### 将上面两个工具整合给下流使用

In [7]:
tools = [search, retriever_tool]

#### 创建代理

首先，我们选择要引导代理的 LLM。

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

接下来，选择要用来指导代理的提示词

In [9]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

现在，我们可以使用 LLM、提示符和工具初始化代理。

Agent负责接收输入并决定采取什么行动。最重要的是，Agent不执行这些操作，这些操作由 AgentExecator (下一步)执行

In [31]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)
# agent.get_prompts()

[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlac

最后，我们将代理(大脑)与 AgentExecator 中的工具结合起来(它将重复调用代理和执行工具)。

In [32]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.agent

RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf

#### 运行代理

我们现在可以在一些查询上运行代理！需要注意，现在这些都是无状态查询(它不会记住以前的交互)。

In [12]:
agent_executor.invoke({"input": "你好!"})



> Entering new AgentExecutor chain...
你好！有什么可以帮助你的吗？

> Finished chain.


{'input': '你好!', 'output': '你好！有什么可以帮助你的吗？'}

In [13]:
agent_executor.invoke({"input": "Es6的promise怎么使用?"})



> Entering new AgentExecutor chain...

Invoking: `es6_promise_search` with `{'query': 'How to use ES6 Promise'}`


ES6 入门教程














back to top
edit
theme
Loading ...
Oops! ... File not found!
上一章下一章




《ECMAScript 6 入门教程》是一本开源的JavaScript语言教程，全面介绍ECMAScript 6新增的语法特性。
本书力争覆盖ES6与ES5的所有不同之处，对涉及的语法知识给予详细介绍，并给出大量简洁易懂的示例代码。
本书为中级难度，适合已有一定JavaScript语言基础的读者，了解这门语言的最新进展；也可当作参考手册，查寻新增的语法点。ES6 Promise是一种用于处理异步操作的对象，它代表一个异步操作的最终完成或失败。要使用ES6 Promise，你可以按照以下步骤进行：

1. 创建一个Promise对象：
```javascript
const myPromise = new Promise((resolve, reject) => {
  // 异步操作
  if (/* 异步操作成功 */) {
    resolve('成功的结果');
  } else {
    reject('失败的原因');
  }
});
```

2. 处理Promise对象的结果：
```javascript
myPromise.then((result) => {
  // 处理成功的结果
  console.log(result);
}).catch((error) => {
  // 处理失败的原因
  console.error(error);
});
```

这样，你就可以使用ES6 Promise来处理异步操作，通过`.then()`方法处理成功的结果，通过`.catch()`方法处理失败的原因。希望这能帮助你更好地理解如何使用ES6 Promise。

> Finished chain.


{'input': 'Es6的promise怎么使用?',
 'output': "ES6 Promise是一种用于处理异步操作的对象，它代表一个异步操作的最终完成或失败。要使用ES6 Promise，你可以按照以下步骤进行：\n\n1. 创建一个Promise对象：\n```javascript\nconst myPromise = new Promise((resolve, reject) => {\n  // 异步操作\n  if (/* 异步操作成功 */) {\n    resolve('成功的结果');\n  } else {\n    reject('失败的原因');\n  }\n});\n```\n\n2. 处理Promise对象的结果：\n```javascript\nmyPromise.then((result) => {\n  // 处理成功的结果\n  console.log(result);\n}).catch((error) => {\n  // 处理失败的原因\n  console.error(error);\n});\n```\n\n这样，你就可以使用ES6 Promise来处理异步操作，通过`.then()`方法处理成功的结果，通过`.catch()`方法处理失败的原因。希望这能帮助你更好地理解如何使用ES6 Promise。"}

In [ ]:
agent_executor.invoke({"input": "北京今天天气怎么样?请用中文回答我"})

#### 增加Memory

##### chat_history

之前的代理是无状态的。这意味着它不记得以前的交互。

为了给它提供内存，我们可以传递一个chat_history。



In [14]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "你好, 我的名字是Barry", "chat_history": []})



> Entering new AgentExecutor chain...
你好，Barry！有什么可以帮助你的吗？

> Finished chain.


{'input': '你好, 我的名字是Barry',
 'chat_history': [],
 'output': '你好，Barry！有什么可以帮助你的吗？'}

In [15]:
from langchain_core.messages import AIMessage, HumanMessage

In [16]:
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="你好, 我的名字是Barry"),
            AIMessage(content="你好，Barry！有什么可以帮助你的吗"),
        ],
        "input": "我的名字是什么?",
    }
)



> Entering new AgentExecutor chain...
您刚刚告诉我您的名字是Barry。您是否想知道您的名字是什么意思或者有其他问题需要帮助解决呢？

> Finished chain.


{'chat_history': [HumanMessage(content='你好, 我的名字是Barry'),
  AIMessage(content='你好，Barry！有什么可以帮助你的吗')],
 'input': '我的名字是什么?',
 'output': '您刚刚告诉我您的名字是Barry。您是否想知道您的名字是什么意思或者有其他问题需要帮助解决呢？'}

##### RunnableWithMessageHistory

除了chat_history方式，我们还可以将消息包装在RunnableWithMessageHistory中

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [18]:
message_history = ChatMessageHistory()

In [19]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 这是必需的，因为在大多数真实场景中，都需要一个会话 ID
    # 这里没有真正使用它，因为我们在内存中使用的是一个简单的 ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [29]:
# 查看agent_with_chat_history的提示词
agent_executor.agent.prompt

AttributeError: 'RunnableMultiActionAgent' object has no attribute 'prompt'

In [24]:
agent_with_chat_history.invoke(
    {"input": "你好, 我的名字是Barry"},
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run f3626dc0-b344-4356-a92f-b4f85d6ecc98 not found for run fc01d3d3-f136-4af9-9765-1fd62e9b5831. Treating as a root run.




> Entering new AgentExecutor chain...
你好，Barry！有什么我可以帮助你的吗？

> Finished chain.


{'input': '你好, 我的名字是Barry',
 'chat_history': [HumanMessage(content='我的名字是什么?'),
  AIMessage(content='很抱歉，我无法直接知道您的名字。您可以告诉我您的名字，这样我就可以称呼您了。您想告诉我您的名字吗？'),
  HumanMessage(content='你好, 我的名字是Barry'),
  AIMessage(content='你好，Barry！有什么我可以帮助你的吗？'),
  HumanMessage(content='我的名字是什么?'),
  AIMessage(content='您刚刚告诉我您的名字是Barry。有什么我可以帮助您的吗？'),
  HumanMessage(content='你好, 我的名字是Barry'),
  AIMessage(content='你好，Barry！有什么我可以帮助你的吗？')],
 'output': '你好，Barry！有什么我可以帮助你的吗？'}

In [22]:
agent_with_chat_history.invoke(
    {"input": "我的名字是什么?"},
    config={"configurable": {"session_id": "<foo>"}},
)

Parent run d4bd4ad8-59cf-4092-a998-9f6b1bb0b40e not found for run 9476e95e-e8ba-4221-b1c9-c938e24ff427. Treating as a root run.




> Entering new AgentExecutor chain...
您刚刚告诉我您的名字是Barry。有什么我可以帮助您的吗？

> Finished chain.


{'input': '我的名字是什么?',
 'chat_history': [HumanMessage(content='我的名字是什么?'),
  AIMessage(content='很抱歉，我无法直接知道您的名字。您可以告诉我您的名字，这样我就可以称呼您了。您想告诉我您的名字吗？'),
  HumanMessage(content='你好, 我的名字是Barry'),
  AIMessage(content='你好，Barry！有什么我可以帮助你的吗？')],
 'output': '您刚刚告诉我您的名字是Barry。有什么我可以帮助您的吗？'}